<h1 style="text-align: center;">Tennis Analyzer ML Model V2</h1>
<h3 style="text-align: center;">Dan Warnick</h3>

<p>To start we will begin by selecting the data points we want to analyze with existing known results. For each data entry we will have two players each with the following data entries.</p>
<table style="font-size: .8em;">
    <tr>
        <th>Player Name</th>
    </tr>
</table>
<p>Along with match facts like Clay/Hard/Grass Court or Indoor/Outdoor. In the future may want to add weather and adjust certain parameters for more accuracy and more data points to train from, however for now this seems a good start.</p>

<h2>1.) Collect Data Efficiently</h2>

In [4]:
import django
from django.http import HttpResponse
from django.template import loader
import os
from django.db import models
import torch
from math import ceil, floor
import math
import numpy as np
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.utils as utils
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
from asgiref.sync import sync_to_async # type: ignore
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score
import copy
import joblib

# os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'breakpoint.settings')
# django.setup()

# from render.models import *

In [5]:
start = '20120101'
end = '20231231'
match_type = 'm'

In [6]:
start_date = datetime.strptime(start, '%Y%m%d').date()
end_date = datetime.strptime(end, '%Y%m%d').date()

# if match_type == 'm':   
#     typer = MensTennisMatch
#     insert_db = MensTennisMatchStats
# else:
#     typer = WomensTennisMatch
#     insert_db = WomensTennisMatchStats

# query = insert_db.objects.filter(
#         tourney_date__range=(start_date, end_date)
#     ).order_by('tourney_date')
    
# games = await sync_to_async(list)(query.all().values())

RD_CUTOFF = 125

# df = pd.read_csv('../../testcsvs/GLICKO6.014.csv')
df = pd.read_csv('../../testcsvs/StephFixRP25.csv')

print(df.columns)
df['tourney_date'] = pd.to_datetime(df['tourney_date']).dt.date
df = df[(df['tourney_date'] >= start_date) & (df['tourney_date'] <= end_date)]
# df = df[(df['a_glicko_rd'] <= RD_CUTOFF) & (df['b_glicko_rd'] <= RD_CUTOFF)]
df = df[(df['a_steph_rd'] <= RD_CUTOFF) & (df['b_steph_rd'] <= RD_CUTOFF)]


df = df.drop(['tourney_id', 'tourney_name', 'match_num', 'tourney_date', 'a_player_name', 'b_player_name', 'a_player_id', 'a_player_slug', 'b_player_id', 'b_player_slug', 'sets', 'games', 'tiebreaks'], axis=1)
# print(df.duplicated().sum())

print(len(df))

one_hot_encoder = OneHotEncoder(sparse_output=False)

# One-Hot Encode the 'category_text' column
category_encoded = one_hot_encoder.fit_transform(df[['surface']])

# Convert to DataFrame
category_encoded_df = pd.DataFrame(category_encoded, columns=one_hot_encoder.get_feature_names_out(['surface']))

# Concatenate the one-hot encoded columns back to the original DataFrame
df = pd.concat([df.reset_index(drop=True), category_encoded_df], axis=1)

# Drop the original 'category_text' column
df.drop('surface', axis=1, inplace=True)
# print(df.duplicated().sum())

print(len(df))

Index(['tourney_id', 'tourney_name', 'tourney_date', 'surface', 'best_of',
       'match_num', 'tourney_level', 'tourney_round', 'a_player_id',
       'a_player_name',
       ...
       'a_surface_return_second_won_steph_rating',
       'b_surface_second_won_steph_rating',
       'a_surface_return_second_won_steph_rd', 'b_surface_second_won_steph_rd',
       'sets', 'games', 'tiebreaks', 'a_odds', 'b_odds', 'a_b_win'],
      dtype='object', length=174)
27357
27357


In [7]:
df.head()

,best_of,tourney_level,tourney_round,a_player_age,a_player_hand,a_player_ht,a_player_rank,a_player_rank_points,b_player_age,b_player_hand,...,a_surface_return_second_won_steph_rating,b_surface_second_won_steph_rating,a_surface_return_second_won_steph_rd,b_surface_second_won_steph_rd,a_odds,b_odds,a_b_win,surface_Clay,surface_Grass,surface_Hard
0,3.0,2.0,0.8,23.0,1.0,180.0,15.0,1925.0,28.0,0.0,...,2214.527572,2200.503515,60.921102,65.322681,1.28,3.59,1.0,0.0,0.0,1.0
1,3.0,2.0,0.8,28.0,1.0,188.0,65.0,755.0,25.0,1.0,...,2212.082683,2195.126051,61.967452,70.349421,NaN,NaN,1.0,0.0,0.0,1.0
2,3.0,2.0,0.8,23.0,1.0,183.0,89.0,612.0,22.0,1.0,...,2192.204097,2221.788831,78.900391,83.442883,2.29,1.59,0.0,0.0,0.0,1.0
3,3.0,2.0,0.8,24.0,1.0,178.0,48.0,915.0,26.0,1.0,...,2204.616997,2199.926824,63.764441,65.658069,2.40,1.54,0.0,0.0,0.0,1.0
4,3.0,2.0,0.8,26.0,1.0,196.0,70.0,693.0,25.0,1.0,...,2170.614312,2201.271946,80.399278,61.103138,4.44,1.19,0.0,0.0,0.0,1.0


In [8]:
import sqlite3
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

# df = df[(df['tourney_level'] == 6.0)]
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('Trained-Model-ML.keras', save_best_only=True, monitor='val_loss', mode='min')

df = df.dropna()

margin = df['a_b_win']
df = df.drop(columns=['a_b_win', 'a_odds', 'b_odds', 'tourney_level'])

data = df.values
data = data.astype(float)

x_train = tf.keras.utils.normalize(data, axis=1)
y_train = np.asarray(margin)

class OutcomeProbability(tf.keras.Model):
    def __init__(self, input_dim=160):
        super(OutcomeProbability, self).__init__()
        hidden_dim1 = round(input_dim / 2)
        hidden_dim2 = round(hidden_dim1 / 2)
        
        self.dropout = layers.Dropout(0.33)
        self.hidden1 = layers.Dense(hidden_dim1, activation='linear')  # No activation here, SiLU will be applied separately
        self.hidden2 = layers.Dense(hidden_dim2, activation='linear')  # No activation here, SiLU will be applied separately
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.relu1 = layers.Activation('silu')  # SiLU in TensorFlow/Keras is swish
        self.relu2 = layers.Activation('silu')
        self.output_layer = layers.Dense(2, activation='softmax')

        
    def call(self, inputs, training=False):
        x = self.dropout(inputs, training=training)
        x = self.bn1(self.relu1(self.hidden1(x)), training=training)
        x = self.bn2(self.relu2(self.hidden2(x)), training=training)
        x = self.output_layer(x)
        return x


model = OutcomeProbability(input_dim=x_train.shape[1])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model training
history = model.fit(x_train, y_train, epochs=100, validation_split=0.1, batch_size=32, callbacks=[earlyStopping, mcp_save])

# Print final accuracy
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Training Accuracy: {train_accuracy:.4f}")
print(f"Final Validation Accuracy: {val_accuracy:.4f}")

print('Done')

Epoch 1/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 830us/step - accuracy: 0.5937 - loss: 0.6917 - val_accuracy: 0.6398 - val_loss: 0.6306
Epoch 2/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.6414 - loss: 0.6291 - val_accuracy: 0.6492 - val_loss: 0.6231
Epoch 3/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - accuracy: 0.6415 - loss: 0.6286 - val_accuracy: 0.6536 - val_loss: 0.6194
Epoch 4/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.6495 - loss: 0.6239 - val_accuracy: 0.6581 - val_loss: 0.6191
Epoch 5/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.6531 - loss: 0.6198 - val_accuracy: 0.6465 - val_loss: 0.6236
Epoch 6/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 0s 658us/step - accuracy: 0.6558 - loss: 0.6175 - val_accuracy: 0.6585 - val_loss: 0.6186
Epoch 7/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - accuracy: 0.6515 - loss: 0.6188 - val_accuracy: 0.6492 - val_loss: 0.6184
Epoch 8/100
632/632 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - accuracy: 0.6581 - loss: 0